# Simple GAN implemented in Keras

## Mount google drive

In [0]:
from google.colab import drive
import os
drive.mount('/content/drive')

## Imports first

In [0]:
import os
import keras
from keras import backend as K
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image as IImage
from PIL import Image
from keras.layers import *
from keras.models import Model, Sequential
from keras.datasets import mnist
import random
import operator
from functools import reduce
from scipy.signal import savgol_filter
%matplotlib inline

## Load and prepare MNIST dataset

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

image_shape = (28, 28, 1)
quarter_image_shape = ()
half_image_shape = ()
if image_shape[2] != 1:
  quarter_image_shape = (image_shape[0] // 4, image_shape[1] // 4, image_shape[2] // 4)
  half_image_shape = (image_shape[0] // 2, image_shape[1] // 2, image_shape[2] // 2)
else:
  quarter_image_shape = (image_shape[0] // 4, image_shape[1] // 4, 1)
  half_image_shape = (image_shape[0] // 2, image_shape[1] // 2, 1)
  
x_real = np.concatenate((x_train, x_test), axis=0)
y_real = np.concatenate((y_train, y_test), axis=0)

def preprocess_imgs(x):
  x = x.reshape((x.shape[0],) + image_shape)
  return x / 255


def preprocess_labels(y):
  return np.ones(y.shape)

x_real = preprocess_imgs(x_real)
y_real = preprocess_labels(y_real)

In [0]:
def make_m_noise_images_of_n_size(m: int, n: int):
  mean = 0
  var = 0.1
  sigma = var**0.5
  return np.random.normal(mean,sigma,(m, n))

batch_size = 256
generator_input_length = np.prod(quarter_image_shape)

## Build model

In [0]:
def build_generator(use_conv=False):
  generator = Sequential()
  
  if use_conv:

    generator.add(
      Dense(np.prod(quarter_image_shape), input_shape=(np.prod(quarter_image_shape),))
    )

    generator.add(
      BatchNormalization()
    )

    generator.add(
      LeakyReLU()
    )

    generator.add(
      Reshape(quarter_image_shape, input_shape=(np.prod(quarter_image_shape),)) 
    )

    generator.add(
      Conv2DTranspose(1, kernel_size=2, strides=2)
    )

    generator.add(Flatten())

    generator.add(
      Dense(np.prod(half_image_shape))
    )

    generator.add(
      BatchNorwmalization()
    )

    generator.add(
      LeakyReLU()
    )

    generator.add(
      Reshape(half_image_shape) 
    )

    generator.add(
      Conv2DTranspose(1, kernel_size=2, strides=2)
    )
  else:
    generator.add(
      Dense(np.prod(quarter_image_shape), input_shape=(np.prod(quarter_image_shape),))
    )
    
    generator.add(
      BatchNormalization()
    )

    generator.add(
      LeakyReLU()
    )
    
    generator.add(
      Dense(np.prod(half_image_shape))
    )
    
    generator.add(
      BatchNormalization()
    )

    generator.add(
      LeakyReLU()
    )
    
    generator.add(
      Dense(np.prod(image_shape))
    )
    
    generator.add(
      BatchNormalization()
    )

    generator.add(
      LeakyReLU()
    )
    
    generator.add(
      Reshape(image_shape) 
    )

  generator.compile(
    'RMSProp', 'mse', ['acc']
  )
  
  return generator

In [0]:
# Create a model which will determine and train image recognition
discriminator = Sequential()
discriminator.add(
    Dense(10, activation='relu', input_shape=image_shape)
)
discriminator.add((Dropout(0.8)))
discriminator.add(
    LeakyReLU(alpha=0.1)
)
discriminator.add((Dropout(0.8)))
discriminator.add(Flatten())
discriminator.add(Dense(128, activation='relu'))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(
  'RMSProp', 'binary_crossentropy', ['acc']
)

# Create a model which will generate images and 
# try to make discriminator give wrong answers

generator = build_generator(use_conv=False)

# Create a model which will train generator
discriminator.trainable = False
gen_and_disc = Sequential()
gen_and_disc.add(generator)
gen_and_disc.add(discriminator)
gen_and_disc.compile(
  'RMSProp', 'binary_crossentropy', ['acc']
)
generator.summary()

W0711 07:50:42.076173 140491410323328 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 49)                2450      
_________________________________________________________________
batch_normalization_3 (Batch (None, 49)                196       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 49)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 196)               9800      
_________________________________________________________________
batch_normalization_4 (Batch (None, 196)               784       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 196)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 784)               154448    
__________

## Stack models and train them

In [0]:
EPOCHS = 1000000
di = 10  # Train discriminator on batch every dith epoch
gadi = 1  # Train generator on batch every gadith epoch


#  this function shuffles xs and ys
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def print_every_nth(i, n):
  if not (i % n):
    print("EPOCH %d" % i)
  
disc_history = {
  'acc': [],
  'loss': []
}

gad_history = {
  'acc': [],
  'loss': []
}
  
for i in range(EPOCHS):
  if not (i % 100):
    print("EPOCH %d" % i)
    plt.figure()
    img = generator.predict(
      make_m_noise_images_of_n_size(1, generator_input_length)
      )[0]
    if image_shape[2] == 1:
      img = np.reshape(img, image_shape[:2])
    plt.imshow(img)
    plt.show()
  #  train discriminator on batch with 50% generated and 50% real
  if not (i % di):
    x_real_temp = x_real[random.sample(range(0, len(x_real)), batch_size // 2)]
    y_real_temp = y_real[random.sample(range(0, len(y_real)), batch_size // 2)]


    x_fake, y_fake = generator.predict(
        make_m_noise_images_of_n_size(batch_size // 2, generator_input_length)
    ), np.zeros(batch_size // 2)

    x, y = unison_shuffled_copies(np.concatenate((x_real_temp, x_fake)), np.concatenate((y_real_temp, y_fake)))

    disc_hist = discriminator.train_on_batch(x, y)
    disc_history['acc'].append(disc_hist[1])
    disc_history['loss'].append(disc_hist[0])
  else:
    disc_history['acc'].append(disc_history['acc'][-1])
    disc_history['loss'].append(disc_history['loss'][-1])
  #  train generator
  if not (i % gadi):
    x, y = make_m_noise_images_of_n_size(batch_size, generator_input_length), np.ones(batch_size)
    gad_hist = gen_and_disc.train_on_batch(x, y)
    gad_history['acc'].append(gad_hist[1])
    gad_history['loss'].append(gad_hist[0])
  else:
    gad_history['acc'].append(gad_history['acc'][-1])
    gad_history['loss'].append(gad_history['loss'][-1])

## Plot losses and accuraccy

In [0]:
print("Loss while training generator and discriminator")
plt.plot(gad_history['loss'], color='r')
plt.plot(disc_history['loss'], color='b')

In [0]:
print("Accuraccy while training generator and discriminator")
plt.plot(gad_history['acc'], color='r')
plt.plot(disc_history['acc'], color='b')

In [0]:
print("Accuraccy while training generator and discriminator")
Y_gad = savgol_filter(gad_history['acc'], 101, 3)
Y_disc = savgol_filter(disc_history['acc'], 101, 3)
plt.plot(Y_gad, color='r')
plt.plot(Y_disc, color='b')

## Show what generator can do

In [0]:
img = generator.predict(
      make_m_noise_images_of_n_size(1, generator_input_length)
  )[0]
if image_shape[2] == 1:
  img = np.reshape(img, image_shape[:2])
plt.imshow(img)